In [269]:
import cv2
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt
from natsort import natsorted
import os
import time
import sys
import copy

**Generate pointcloud from the disparity maps and filter the pointcloud by depth.**

In [281]:
disparity_dir = '../../optimization_data/disp'
save_dir = '../../optimization_data/pointcloud'
Q = np.load('../params/Q.npy')
count = 0
start = time.time()
# Define the depth threshold
depth_threshold = 0.25 # meters

# Sequence of images from the left/right dir
disparities = natsorted([os.path.join(disparity_dir, f) for f in os.listdir(disparity_dir) if ".npy" in f])

for disp_map in disparities:
    disp = np.load(disp_map)
    point_cloud = cv2.reprojectImageTo3D(disp, Q)
    point_cloud = point_cloud.reshape(-1, 3)

    # Filter out the points beyond the depth threshold
    filtered_point_cloud = point_cloud[abs(point_cloud[:, 2]) < abs(depth_threshold)]

    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(filtered_point_cloud)
    # o3d.visualization.draw_geometries([pcd])
    save_path = os.path.join(save_dir, f'{count}.ply')
    o3d.io.write_point_cloud(save_path, pcd)
    print('Finished saving: '+f'{count}.ply')
    
    count += 1
end = time.time()
print(f'Done with pointcloud saving in {end - start}s')

[[ 1.00000000e+00  0.00000000e+00  0.00000000e+00 -9.36040771e+02]
 [ 0.00000000e+00  1.00000000e+00  0.00000000e+00 -5.11093964e+02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.44359363e+03]
 [ 0.00000000e+00  0.00000000e+00 -1.58730159e+01  0.00000000e+00]]
Finished saving: 0.ply
Finished saving: 1.ply
Finished saving: 2.ply
Finished saving: 3.ply
Finished saving: 4.ply
Finished saving: 5.ply
Finished saving: 6.ply
Finished saving: 7.ply
Finished saving: 8.ply
Finished saving: 9.ply
Finished saving: 10.ply
Finished saving: 11.ply
Finished saving: 12.ply
Finished saving: 13.ply
Finished saving: 14.ply
Finished saving: 15.ply
Finished saving: 16.ply
Finished saving: 17.ply
Finished saving: 18.ply
Finished saving: 19.ply
Finished saving: 20.ply
Finished saving: 21.ply
Finished saving: 22.ply
Finished saving: 23.ply
Finished saving: 24.ply
Finished saving: 25.ply
Finished saving: 26.ply
Finished saving: 27.ply
Finished saving: 28.ply
Finished saving: 29.ply
Finished saving: 30.p

KeyboardInterrupt: 

In [248]:
# pcd = o3d.io.read_point_cloud('../data/phantom_point-cloud_data/phacon_exp_3.ply')
# o3d.visualization.draw_geometries([pcd])

**Do phantom registration for all pointcloud**

In [258]:
sys.path.append('../')
sys.path.insert(0, '../optical_tracking')
sys.path.insert(0, '../util')
from phantom_registration import manual_registration, draw_registration_result
from Solver import solver
sol = solver()

In [286]:
def phantom_registration(pointcloud_dir):

    pcd_source_init = o3d.io.read_point_cloud("../../optimization_data/pointcloud/33.ply")
    pcd_target = o3d.io.read_point_cloud("../data/phantom_point-cloud_data/phacon_exp_3.ply")
    pcd_target = pcd_target.voxel_down_sample(voxel_size=0.001)
    cam2ph_list = []

    # manual registration to initialize
    result = manual_registration(pcd_source_init, pcd_target)
    pointclouds = natsorted([os.path.join(pointcloud_dir, f) for f in os.listdir(pointcloud_dir) if ".ply" in f])

    print("Apply point-to-plane ICP")

    for pointcloud in pointclouds:
        pcd_source = o3d.io.read_point_cloud(pointcloud)
        
        o3d.visualization.draw_geometries([pcd_target, pcd_source])
        break
        threshold = 0.0008

        trans_init_icp = result.transformation

        result = o3d.pipelines.registration.registration_icp(
            pcd_source, pcd_target, threshold, trans_init_icp,
            o3d.pipelines.registration.TransformationEstimationPointToPlane())

        print("Transformation is:")
        print(result.transformation)
        draw_registration_result(pcd_source, pcd_target, result.transformation)
        ph2cam = result.transformation
        cam2ph = sol.invTransformation(ph2cam)
        cam2ph_list.append(cam2ph)
        
        T_o_p = np.array(cam2ph_list)
        print(T_o_p.shape)
        if count == 5:
            break

    T_o_p = np.array(cam2ph_list)
    # np.save('T_o_p.npy', T_o_p)


In [289]:
pointcloud_dir = '../../optimization_data/pointcloud/'
phantom_registration(pointcloud_dir)

Visualization of two point clouds before manual alignment

1) Please pick at least three correspondences using [shift + left click]
   Press [shift + right click] to undo point picking
2) After picking points, press 'Q' to close the window


1) Please pick at least three correspondences using [shift + left click]
   Press [shift + right click] to undo point picking
2) After picking points, press 'Q' to close the window



AssertionError: 